# Step 2
## Visualising the data

This notebook shows how I would start to visualise the data.

There are some thoughts about visualisation techniques along the way.

In [ ]:
"""
Import some useful modules
"""

from pathlib import Path

import numpy as np
import SimpleITK as sitk

import matplotlib.pyplot as plt

from platipy.imaging import ImageVisualiser
from platipy.dicom.io.crawl import process_dicom_directory
from platipy.imaging.label.utils import get_com
from platipy.imaging.utils.crop import crop_to_label_extent

import seaborn as sns

from scipy.stats import gaussian_kde
%matplotlib notebook

In [ ]:
"""
Let's load a single image with the segmentations 
"""

input_dir = Path("./input/NIfTI/RTMAC_LIVE_001/")

img = sitk.ReadImage( str(list(input_dir.glob("IMAGES/*.nii.gz"))[0]) , sitk.sitkUInt32)
contours = {
    s_file.name[26:-7]:sitk.ReadImage( str(s_file) )
    for s_file in input_dir.glob("STRUCTURES/*.nii.gz")
}

In [ ]:
"""
Before we start to visualise, let's look at the imaging data
Here we convert the image to a 1D array (list) of values
This makes it pretty efficient to pass to other functions
"""

img_values = sitk.GetArrayFromImage(img).flatten()

In [ ]:
"""
We can plot a histogram of intensity
This gives us some useful information we will use shortly
"""

fig, ax = plt.subplots(1,1)
ax.hist(img_values)
fig.show()

### Can we make this plot a bit nicer?

Some thoughts:
 - not enough information to show this to another person (i.e. myself in the future)
 - the plot isn't particularly informative
 - the default colour is a bit plain

In [ ]:
"""
Maybe a bit nicer
"""

# By creating a matplotlib axis we have a lot more control over the plot
# There are lot's of ways to do this:
# fig = plt.figure(); ax = fig.add_subplots(1,1)
fig, ax = plt.subplots(1,1,figsize=(6,5))

# We are going to use the same function as before
# This time we will select the bins 
# And use a "step" histogram to avoid filling in too much space
ax.hist(
    img_values,
    bins=np.linspace(0,450,50),
    histtype="step",
    ec="#34495E",
    lw=1.5,
    label="Entire Image",
)

# Now we are going to plot histograms of the image intensity for the contours
# This tells us about the "interesting" parts of the image
# Usually this is a good way to choose windowing levels

# What colours should you use?
# Here, whatever you like! I prefer muted colours
for s, color in zip(contours, sns.color_palette("gist_earth", n_colors=8)):
    
    # We don't want to plot too many structures, so skip all the left-sided ones
    if "_L" not in s:
        continue
    
    # Extract out the image where the contour is defined
    s_vals = sitk.GetArrayViewFromImage(img)[np.where(sitk.GetArrayViewFromImage(contours[s]))]
    
    # Same as before - by using the same bins the plots will use a bit nicer
    ax.hist(
        s_vals,
        bins=np.linspace(0,450,50),
        histtype="stepfilled",
        fc=color,
        ec=color,
        alpha=0.5,
        lw=1.5,
        label=s
    )

# A logarithmic axis could work
# Here I chose a "symlog" (symmetric log)
# This is good when the function being plotted gets close to zero
ax.set_yscale("symlog", linthresh=100)

# Not much point going beyond the bins
ax.set_xlim(0,450)

# !
ax.set_xlabel("Image Value")
ax.set_ylabel("Number of Voxels")

ax.grid()
ax.set_axisbelow(True)

ax.legend()

fig.tight_layout()

fig.show()

In [ ]:
"""
Another good way to represent distributions is the a kernel density estimate
"""

# We can use much finer spacing
x = np.linspace(0,450,500)

fig, ax = plt.subplots(1,1,figsize=(6,5))

kde_fit = gaussian_kde(img_values[::1000])

ax.plot(
    x,
    kde_fit(x),
    c="#34495E",
    lw=2,
    label="Entire Image",
)


for s, color in zip(contours, sns.color_palette("gist_earth", n_colors=8)):
    
    
    if "_L" not in s:
        continue
    
    
    s_vals = sitk.GetArrayViewFromImage(img)[np.where(sitk.GetArrayViewFromImage(contours[s]))]
    
    
    kde_fit = gaussian_kde(s_vals[::50])

    ax.plot(
        x,
        kde_fit(x),
        c=color,
        lw=2,
        label=s,
    )


ax.set_xlim(0,450)
ax.set_ylim(0,0.025)

ax.set_xlabel("Image Value")
ax.set_ylabel("Relative Frequency")

ax.grid()
ax.set_axisbelow(True)

ax.legend()

fig.tight_layout()

fig.show()

### Platipy has some really cool image visualisation tools

There are functions for *most* things you'd like to visualise

But if you can't find it... code it up and make a pull request!

In [ ]:
"""
Let's visualise with some knowledge of the intensities
"""

vis = ImageVisualiser(img, window=(0,400), figure_size_in=6)
fig = vis.show()

In [ ]:
fig.savefig("mri.jpeg", dpi=300)

In [ ]:
"""
We can also display contours on top of images
"""

vis = ImageVisualiser(img, window=(0,400), cut=(40,250,250), figure_size_in=6)

vis.add_contour(contours)

fig = vis.show()

In [ ]:
"""
We can also overlay scalar maps
Here is an example of an overlay
that only maps part of the image
"""

def super_complex_ai_algorithm(img, contour):
    
    img_masked = crop_to_label_extent(img, contour)
    
    ai_result = sitk.Cast(img_masked, sitk.sitkFloat32)**0.5
    
    values = sitk.GetArrayViewFromImage(ai_result)
    
    ai_mean = values.mean()
    ai_sigma = values.std()
    
    ai_output = sitk.Abs(ai_result - ai_mean)/ai_sigma
    
    return sitk.Resample(ai_output, img)
                  

ai_output = super_complex_ai_algorithm(img, contours["PAROTID_L"])

In [ ]:
ai_output

In [ ]:
"""
A preliminary visualisation
"""

vis = ImageVisualiser(img, window=(0,400), cut=get_com(contours["PAROTID_L"]), figure_size_in=6)

vis.add_scalar_overlay(ai_output)

fig = vis.show()

In [ ]:
"""
Some minor modifications
"""

vis = ImageVisualiser(img, window=(0,400), cut=get_com(contours["PAROTID_L"]), figure_size_in=6)

vis.add_scalar_overlay(
    scalar_image = ai_output,
    name="AI results [units]",
    colormap=plt.cm.magma,
    alpha=0.75,
    min_value=0,
    max_value=2,
    discrete_levels=8,
    mid_ticks=False,
    show_colorbar=True,
    norm=None,
)

vis.add_bounding_box(contours["PAROTID_L"], color="#29bd34", linewidth=3, name="VOI")

vis.set_limits_from_label(contours["PAROTID_L"], expansion=50)

fig = vis.show()

### Projection images

These can be really useful to see the extents of contours.

And to quickly spot errors!

There are a few different types I'll go through.

In [ ]:
"""
There are a few ways to do this - the easiest is with ImageVisualiser

First let's have a look at the mean intensity projection
"""

# The intensity is normalised to [0,1]
vis = ImageVisualiser(img, window=(0,0.8), projection="mean", figure_size_in=6)

fig = vis.show()

In [ ]:
"""
We can also add contours
"""

vis = ImageVisualiser(img, window=(0,0.8), projection="mean")

vis.add_contour(contours)

fig = vis.show()

In [ ]:
fig.savefig("./figures/synthetic_mri_drr.jpeg", dpi=300)